Reproduce KCMI evaluation from ManTraNet

In [1]:
import torch
from torch import nn
from importlib import reload
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from prettytable import PrettyTable

from utils import stratified_train_val_test_split, get_ela
from sklearn.model_selection import train_test_split

import albumentations
from albumentations import augmentations
import albumentations.pytorch

In [2]:
from segmentation.merged_net import SRM_Classifer

device = 'cuda'

model = SRM_Classifer(encoder_checkpoint='best_weights/pretrain_[31_03_12_16_32].h5', num_classes=312).to(device)
model.eval();

--------- Loaded Checkpoint: best_weights/pretrain_[31_03_12_16_32].h5 ----------
<All keys matched successfully>


In [3]:
df = pd.read_csv('kcmi.csv')

df_train, df_test = train_test_split(df, test_size=0.07, shuffle=True, random_state=123)

In [4]:
print(df_train.shape)
# print(df_val.shape)
print(df_test.shape)

(2301, 3)
(174, 3)


In [5]:
def get_tensor(image, ela_image):

    #---------------- Reshape & Normalize -----------------------
    image = augmentations.geometric.functional.resize(image, 256, 256, cv2.INTER_AREA)
    ela_image = augmentations.geometric.functional.resize(ela_image, 256, 256, cv2.INTER_AREA)

    normalize = {
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225],
    }

    transforms_normalize = albumentations.Compose(
        [
            albumentations.Normalize(mean=normalize['mean'], std=normalize['std'], always_apply=True, p=1),
            albumentations.pytorch.transforms.ToTensorV2()
        ],
        additional_targets={'ela':'image'}
    )

    data = transforms_normalize(image=image, ela=ela_image)
    image_tensor = data["image"].unsqueeze(0)
    ela_tensor = data["ela"].unsqueeze(0)
    
    return image_tensor, ela_tensor

In [6]:
def get_data(df):
    X, Y = [], []

    for row in tqdm(df.values):
        image_path, class_id, label = row

        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        x = image.shape[0]//2 - 256
        y = image.shape[1]//2 - 256

        patch_image = image[x:x+512, y:y+512, :]
        patch_ela = get_ela(patch_image, 25)

        # plt.figure()
        # plt.subplot(121)
        # plt.imshow(patch_image)
        # plt.subplot(122)
        # plt.imshow(patch_ela)
        # plt.show()

        image_tensor, ela_tensor = get_tensor(patch_image, patch_ela)
        
        _, (reduced_feat, _, _, _, _) = model(image_tensor.to(device), ela_tensor.to(device))
        reduced_feat = reduced_feat.cpu().detach().numpy()
        
        X.append(reduced_feat)
        Y.append(label)

    return X, Y

In [7]:
trainX, trainY = get_data(df_train)
testX, testY = get_data(df_test)

100%|██████████| 174/174 [01:08<00:00,  2.53it/s]


In [8]:
X_train = np.array([x.reshape(-1) for x in trainX])
y_train = np.array([trainY]).T.ravel()

X_test = np.array([x.reshape(-1) for x in testX])
y_test = np.array([testY]).T.ravel()

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2301, 256) (2301,)
(174, 256) (174,)


# KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7, n_jobs=-1)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

In [10]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.2413793103448276


# SVM

In [13]:
import SVM
reload(SVM)

hyper_params = [{
    'kernel': ['rbf', 'poly', 'sgmoid'], 
    'gamma': [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 'scale', 'auto'], 
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 3000, 5000, 10000, 30000, 50000, 100000]
}]

opt_params = SVM.optimize_hyperparams(X_train, y_train, params=hyper_params)

Fitting 10 folds for each of 312 candidates, totalling 3120 fits
/home/sowmitra/anaconda3/envs/dfdcpy37env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.11516657 0.25553924        nan 0.11516657 0.26814417        nan
 0.11516657 0.20295502        nan 0.11516657 0.11951252        nan
 0.11516657 0.11299266        nan 0.11516657 0.11429701        nan
 0.11516657 0.11516657        nan 0.11516657 0.12430077        nan
 0.11516657 0.25553924        nan 0.11516657 0.25597403        nan
 0.11516657 0.2542349         nan 0.11516657 0.11864672        nan
 0.11516657 0.11559947        nan 0.11516657 0.11473179        nan
 0.11516657 0.11429701        nan 0.11516657 0.18600791        nan
 0.11516657 0.25553924        nan 0.11516657 0.25553924        nan
 0.20905327 0.2733371         nan 0.20817241 0.11777527        nan
 0.13255035 0.11646716        nan 0.11516657 0.11169019        nan
 0.11516657 0.11299454      

In [14]:
print(opt_params)
SVM.classify(X_train, y_train, opt_params)

{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    1.0s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    1.0s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    1.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.1s finished
[0.27705628 0.29130435 0.26086957 0.29565217 0.33478261 0.28695652
 0.2826087  0.23043478 0.2826087  0.35217391]
0.2894447581404103
0.03253353008771719
